In [82]:
#https://blog.paperspace.com/building-double-deep-q-network-super-mario-bros/
from IPython import display
!pip install nes-py==0.2.6
!pip install gym-super-mario-bros
!apt-get update
!apt-get install ffmpeg libsm6 libxext6  -y
!apt install -y libgl1-mesa-glx
!pip install opencv-python

  Using cached nes_py-0.2.6-cp39-cp39-macosx_10_9_x86_64.whl
  Attempting uninstall: nes-py
    Found existing installation: nes-py 8.1.8
    Uninstalling nes-py-8.1.8:
      Successfully uninstalled nes-py-8.1.8
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gym-super-mario-bros 7.3.0 requires nes-py>=8.0.0, but you have nes-py 0.2.6 which is incompatible.
You should consider upgrading via the '/Users/23zhou/PycharmProjects/JupyterMario/venv/bin/python -m pip install --upgrade pip' command.
  Using cached nes_py-8.1.8-cp39-cp39-macosx_10_9_x86_64.whl
  Attempting uninstall: nes-py
    Found existing installation: nes-py 0.2.6
    Uninstalling nes-py-0.2.6:
      Successfully uninstalled nes-py-0.2.6
You should consider upgrading via the '/Users/23zhou/PycharmProjects/JupyterMario/venv/bin/python -m pip install --upgrade pip' command.
/bin/bash: apt-get: comma

In [83]:
import torch
import torch.nn as nn
import random
import gym_super_mario_bros
from nes_py.wrappers import JoypadSpace
from tqdm import tqdm
import pickle
from gym_super_mario_bros.actions import RIGHT_ONLY
import gym
import numpy as np
import collections
import cv2
import matplotlib.pyplot as plt

In [84]:
class MaxandSkipEnv(gym.Wrapper):
    def __init__(self, env=None,skip=4):
        'return every 4th frame'
        super(MaxandSkipEnv, self).__init__(env)
        self._obs_buffer = collections.deque(maxlen=2)
        self._skip = skip

    def step(self, action):
        total_reward = 0.0
        done = None
        for _ in range(self._skip):
            obs, reward, done, info = self.env.step(action)
            self._obs_buffer.append(obs)
            total_reward += reward
            if done:
                break
        max_frame = np.max(np.stack(self._obs_buffer), axis=0)
        return max_frame, total_reward, done, info

    def reset(self):
        "Clears frames"
        self._obs_buffer.clear()
        obs = self.env.reset()
        self._obs_buffer.append(obs)
        return obs

class ProcessFrame84(gym.ObservationWrapper):
    "returns downsized image to 84x84, grayscaled"
    def __init__(self,env=None):
        super(ProcessFrame84,self).__init__(env)
        self.observation_space = gym.spaces.Box(low=0, high=255, shape=(84, 84, 1), dtype=np.uint8)

    def observation(self, obs):
        return ProcessFrame84.process(obs)

    @staticmethod
    def process(frame):
        if frame.size == 240 * 256 * 3:
            img = np.reshape(frame, [240, 256, 3]).astype(np.float32)
        else:
            assert False, "Unknown resolution."
        img = img[:, :, 0] * 0.299 + img[:, :, 1] * 0.587 + img[:, :, 2] * 0.114
        resized_screen = cv2.resize(img, (84, 110), interpolation=cv2.INTER_AREA)
        x_t = resized_screen[18:102, :]
        x_t = np.reshape(x_t, [84, 84, 1])
        return x_t.astype(np.uint8)

class ImageToPyTorch(gym.ObservationWrapper):
    def __init__(self, env):
        super(ImageToPyTorch, self).__init__(env)
        old_shape = self.observation_space.shape
        self.observation_space = gym.spaces.Box(low=0.0, high=1.0, shape=(old_shape[-1], old_shape[0], old_shape[1]),
                                                dtype=np.float32)
    def observation(self, observation):
        return np.moveaxis(observation,2,0)

class ScaledFloatFrame(gym.ObservationWrapper):

    def observation(self, obs):
        return np.array(obs).astype(np.float32) / 255.0

class BufferWrapper(gym.ObservationWrapper):
    def __init__(self,env,n_steps,dtype=np.float32):
        super(BufferWrapper,self).__init__(env)
        self.dtype = dtype
        oldSpace = env.observation_space
        self.observation_space = gym.spaces.Box(oldSpace.low.repeat(n_steps, axis=0),
                                                oldSpace.high.repeat(n_steps, axis=0),
                                                dtype=dtype)
    def reset(self):
        self.buffer = np.zeros_like(self.observation_space.low, dtype=self.dtype)
        return self.observation(self.env.reset())

    def observation(self, observation):
        self.buffer[:-1] = self.buffer[1:]
        self.buffer[-1] = observation
        return self.buffer

In [85]:
def make_env(env):
    env = MaxandSkipEnv(env)
    env = ProcessFrame84(env)
    env = ImageToPyTorch(env)
    env = BufferWrapper(env,4)
    env = ScaledFloatFrame(env)
    return JoypadSpace(env,RIGHT_ONLY)


In [86]:
class DQNSolver(nn.Module):

    def __init__(self, input_shape, n_actions):
        super(DQNSolver, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(input_shape[0], 32, kernel_size=8, stride=4),
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=4, stride=2),
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3, stride=1),
            nn.ReLU()
        )

        conv_out_size = self._get_conv_out(input_shape)
        self.fc = nn.Sequential(
            nn.Linear(conv_out_size, 512),
            nn.ReLU(),
            nn.Linear(512, n_actions)
        )

    def _get_conv_out(self, shape):
        o = self.conv(torch.zeros(1, *shape))
        return int(np.prod(o.size()))

    def forward(self, x):
        conv_out = self.conv(x).view(x.size()[0], -1)
        return self.fc(conv_out)


In [87]:
class DQNAgent:

    def __init__(self, stateSpace, actionSpace, maxMemorySize, batchSize, gamma, lr, dropout, explorationMax, explorationMin, explorationDecay, doubleDQ, pretrained):

        #Defines DQN Layers
        self.stateSpace = stateSpace
        self.actionSpace = actionSpace
        self.doubleDQ = doubleDQ
        self.pretrained = pretrained
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
        if self.doubleDQ:
            self.local_net = DQNSolver(stateSpace,actionSpace).to(self.device)
            self.target_net = DQNSolver(stateSpace,actionSpace).to(self.device)

            if self.pretrained:
                self.local_net.load_state_dict(torch.load("./betterMarioModels/dq1.pt", map_location=torch.device(self.device)))
                self.target_net.load_state_dict(torch.load("./betterMarioModels/dq2.pt", map_location=torch.device(self.device)))

            self.optimizer = torch.optim.Adam(self.local_net.parameters(), lr=lr)
            self.copy = 5000 #steps until copy local weights
            self.step = 0

        else:
            self.dqn = DQNSolver(stateSpace, actionSpace).to(self.device)

            if self.pretrained:
                self.dqn.load_state_dict(torch.load("./betterMarioModels/dq.pt", map_location=torch.device(self.device)))
            self.optimizer = torch.optim.Adam(self.dqn.parameters(), lr=lr)

        self.max_memory_size = maxMemorySize
        if self.pretrained:
            self.STATE_MEM = torch.load("./betterMarioModels/STATE_MEM.pt")
            self.ACTION_MEM = torch.load("./betterMarioModels/ACTION_MEM.pt")
            self.REWARD_MEM = torch.load("./betterMarioModels/REWARD_MEM.pt")
            self.STATE2_MEM = torch.load("./betterMarioModels/STATE2_MEM.pt")
            self.DONE_MEM = torch.load("./betterMarioModels/DONE_MEM.pt")
            with open("ending_position.pkl", 'rb') as f:
                self.ending_position = pickle.load(f)
            with open("num_in_queue.pkl", 'rb') as f:
                self.num_in_queue = pickle.load(f)
        else:
            self.STATE_MEM = torch.zeros(maxMemorySize, *self.stateSpace)
            self.ACTION_MEM = torch.zeros(maxMemorySize, 1)
            self.REWARD_MEM = torch.zeros(maxMemorySize, 1)
            self.STATE2_MEM = torch.zeros(maxMemorySize, *self.stateSpace)
            self.DONE_MEM = torch.zeros(maxMemorySize, 1)
            self.ending_position = 0
            self.num_in_queue = 0

        self.memory_sample_size = batchSize

        #learning parameters
        self.gamma = gamma
        #Huber loss
        self.l1 = nn.SmoothL1Loss().to(self.device)
        self.exploration_max = explorationMax
        self.exploration_min = explorationMin
        self.exploration_rate = explorationMax
        self.exploration_decay = explorationDecay

    def remember(self, state, action, reward, state2, done):
        self.STATE_MEM[self.ending_position] = state.float()
        self.ACTION_MEM[self.ending_position] = action.float()
        self.REWARD_MEM[self.ending_position] = reward.float()
        self.STATE2_MEM[self.ending_position] = state2.float()
        self.DONE_MEM[self.ending_position] = done.float()
        # FIFO tensor
        self.ending_position = (self.ending_position + 1) % self.max_memory_size
        self.num_in_queue = min(self.num_in_queue + 1, self.max_memory_size)

    def recall(self):
        #randomly sample batches
        idx = random.choices(range(self.num_in_queue), k=self.memory_sample_size)

        STATE = self.STATE_MEM[idx]
        ACTION = self.ACTION_MEM[idx]
        REWARD = self.REWARD_MEM[idx]
        STATE2 = self.STATE2_MEM[idx]
        DONE = self.DONE_MEM[idx]

        return STATE, ACTION, REWARD, STATE2, DONE

    def act(self,state):
        #Epsilon-greedy action

        if self.doubleDQ:
            self.step += 1
        if random.random() < self.exploration_rate:
            return torch.tensor([[random.randrange(self.actionSpace)]])
        if self.doubleDQ:
            #use local net for policy if doubleDQ
            return torch.argmax(self.local_net(state.to(self.device))).unsqueeze(0).unsqueeze(0).cpu()
        else:
            return torch.argmax(self.dqn(state.to(self.device))).unsqueeze(0).unsqueeze(0).cpu()

    def copy_model(self):
        #copy model weights into target net
        self.target_net.load_state_dict(self.local_net.state_dict())

    def experience_replay(self):

        if self.doubleDQ and self.step % self.copy == 0:
            self.copy_model()

        if self.memory_sample_size > self.num_in_queue:
            return

        STATE, ACTION, REWARD, STATE2, DONE = self.recall()
        STATE = STATE.to(self.device)
        ACTION = ACTION.to(self.device)
        REWARD = REWARD.to(self.device)
        STATE2 = STATE2.to(self.device)
        DONE = DONE.to(self.device)

        self.optimizer.zero_grad()

        if self.doubleDQ:
            #Double Q-Learning target is Q*(S, A) <- r + γ max_a Q_target(S', a)
            target = REWARD + torch.mul((self.gamma *
                                        self.target_net(STATE2).max(1).values.unsqueeze(1)),
                                        1 - DONE)
            current = self.local_net(STATE).gather(1, ACTION.long()) # Local net approximation of Q-value
        else:
            # Q-Learning target is Q*(S, A) <- r + γ max_a Q(S', a)
            target = REWARD + torch.mul((self.gamma *
                                        self.dqn(STATE2).max(1).values.unsqueeze(1)),
                                        1 - DONE)
            current = self.dqn(STATE).gather(1, ACTION.long())

        loss = self.l1(current, target)
        #gradient
        loss.backward()
        #backprogogation error
        self.optimizer.step()
        self.exploration_rate *= self.exploration_decay

        # Makes sure that exploration rate is always at least 'exploration min'
        self.exploration_rate = max(self.exploration_rate, self.exploration_min)




In [88]:
def vectorize_action(action, action_space):
    #given a scalar action, returns a one-hot encoded action
    return [0 for _ in range(action)] + [1] + [0 for _ in range(action +1, action_space)]



In [89]:
def show_state(env, ep=0, info=""):
    plt.figure(3)
    plt.clf()
    plt.imshow(env.render(mode='rgb_array'))
    plt.title("Episode: %d %s" % (ep, info))
    plt.axis('off')

    display.clear_output(wait=True)
    display.display(plt.gcf())


In [90]:
def run(trainingMode,preTrained):

    env = gym_super_mario_bros.make('SuperMarioBros-1-1-v0')
    env = make_env(env)
    observationSpace = env.observation_space.shape
    actionSpace = env.action_space.n
    agent = DQNAgent(stateSpace=observationSpace,
                     actionSpace=actionSpace,
                     maxMemorySize=30000,
                     batchSize= 32,
                     gamma= 0.90,
                     lr = 0.00025,
                     dropout= 0.,
                     explorationMax= 1.0,
                     explorationMin=0.02,
                     explorationDecay= 0.99,
                     doubleDQ= True,
                     pretrained= preTrained)

    numEpisodes = 6000
    env.reset()
    totalRewards = []

    for epNum in tqdm(range(numEpisodes)):
        state = env.reset()
        state = torch.Tensor([state])
        totalReward = 0
        steps = 0
        while True:
            if not trainingMode:
                show_state(env, epNum)
            action = agent.act(state)
            steps += 1

            state_next, reward, terminal, info = env.step(int(action[0]))
            totalReward += reward
            state_next = torch.Tensor([state_next])
            reward = torch.tensor([reward]).unsqueeze(0)

            terminal = torch.tensor([int(terminal)]).unsqueeze(0)

            if trainingMode:
                agent.remember(state, action, reward, state_next, terminal)
                agent.experience_replay()

            state = state_next
            if terminal:
                break

        totalRewards.append(totalReward)

        print("Total reward after episode {} is {}".format(epNum +1, totalRewards[-1]))
        numEpisodes += 1

    if trainingMode:
        with open("ending_position.pkl", "wb") as f:
            pickle.dump(agent.ending_position, f)
        with open("num_in_queue.pkl", "wb") as f:
            pickle.dump(agent.num_in_queue, f)
        with open("total_rewards.pkl", "wb") as f:
            pickle.dump(totalRewards, f)
        if agent.doubleDQ:
            torch.save(agent.local_net.state_dict(), "./betterMarioModels/dq1.pt")
            torch.save(agent.target_net.state_dict(), "./betterMarioModels/dq2.pt")
        else:
            torch.save(agent.dqn.state_dict(), "./betterMarioModels/dq.pt")
        torch.save(agent.STATE_MEM,  "./betterMarioModels/STATE_MEM.pt")
        torch.save(agent.ACTION_MEM, "./betterMarioModels/ACTION_MEM.pt")
        torch.save(agent.REWARD_MEM, "./betterMarioModels/REWARD_MEM.pt")
        torch.save(agent.STATE2_MEM, "./betterMarioModels/STATE2_MEM.pt")
        torch.save(agent.DONE_MEM,   "./betterMarioModels/DONE_MEM.pt")

    env.close()

    if numEpisodes > 500:
        plt.title("Episodes trained vs. Average Rewards (per 500 eps)")
        plt.plot([0 for _ in range(500)] +
                 np.convolve(totalRewards,np.ones((500,))/500, mode = "valid").tolist())
        plt.show()


In [91]:
run(trainingMode = True, preTrained = False )


  0%|          | 1/6000 [00:13<22:13:51, 13.34s/it]

Total reward after episode 1 is 602.0


  0%|          | 2/6000 [00:19<15:18:29,  9.19s/it]

Total reward after episode 2 is 226.0


  0%|          | 3/6000 [00:24<11:40:58,  7.01s/it]

...

Total reward after episode 387 is 1698.0


  6%|▋         | 388/6000 [1:41:08<27:25:31, 17.59s/it]

Total reward after episode 388 is 1050.0


  6%|▋         | 388/6000 [1:41:21<24:26:05, 15.67s/it]


KeyboardInterrupt: 